In [1]:
import pandas as pd
from tqdm import tqdm

# Read the CSV file
df = pd.read_csv("top_k_prediction/merged_predictions_latest.csv")

# Extract the desired columns
columns = ["id", "top-k", "alternatives","Final Prediction"]
df_labels = df[columns]


In [2]:
import pandas as pd
from tqdm import tqdm

# Read the CSV file
df = pd.read_csv("public_data/wi_dataset_processed_en.csv")

# Extract the desired columns
columns = ["id", "translated_title", "final_description"]
df_desc = df[columns]


In [3]:
merged_df = df_labels.merge(df_desc, on='id')
merged_df

,id,top-k,alternatives,Final Prediction,translated_title,final_description
0,872828466,"7132, 7131, 7231, 9122, 7316, 7232, 3139, 3115...","7131, 7231, 9122",7132,Panel & Paint Technician,Panel & Paint Technician Panel & Paint Technic...
1,839465958,"2320, 7533, 7318, 2330, 2353, 7222, 8152, 2355...",NaN,2330,Teacher in crafts and technology for grades 7-...,Teacher in crafts and technology for grades 7-...
2,857077872,"2211, 2212, 2240, 3258, 2230, 4229, 3211, 4221...",NaN,2212,Consultants in Emergency Medicine - Doughiska,Consultants in Emergency Medicine - Doughiska ...
3,801801567,"3512, 2522, 2356, 3511, 2511, 2153, 1330, 3513...",NaN,3512,Senior IT Support Engineers,"Senior IT Support Engineers My Client, who has..."
4,855162927,"3322, 2433, 2434, 5243, 5223, 5244, 5242, 1221...",NaN,3322,Commercial Sales Representatives,"Commercial Sales Representatives Jobbtitel: ""C..."
...,...,...,...,...,...,...
25660,862998979,"3433, 5223, 5243, 5242, 5211, 3432, 3322, 5244...",NaN,5223,Gallerato showroom sales consultant,Gallerato showroom sales consultant Descriptio...
25661,793143661,"9312, 9313, 9622, 9510, 8343, 9331, 9214, 8342...","9510, 9613, 9329",5419,Pedestrian Marshall,Pedestrian Marshall Pedestrial Marshall / Yard...
25662,725881734,"2513, 2519, 2512, 2166, 2514, 2431, 3435, 2522...",NaN,2166,Unity Gameplay Developer (f/m/d),Unity Gameplay Developer (f/m/d) Unity Gamepla...
25663,880528881,"7115, 7522, 7111, 9313, 7119, 7121, 7123, 9312...",NaN,7115,Carpenter 4505717 | careers4a.com,Carpenter 4505717 | careers4a.com What: jo...


In [4]:
import pandas as pd
from tqdm import tqdm

# Read the CSV file
df = pd.read_csv("enriched_data/wi_labels_enriched.csv")

# Extract the desired d
columns = ["code", "description", "english title"]
df_code = df[columns]

In [5]:
df_code['english title'] = df_code['english title'].str.split(';')
df_code_exploded = df_code.explode('english title')
df_code_exploded['english title'] = df_code_exploded['english title'].str.lower().str.replace(":", "").str.replace(",", "").str.strip()
df_code_exploded

/tmp/ipykernel_1135418/1286929069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_code['english title'] = df_code['english title'].str.split(';')


,code,description,english title
0,110,Commissioned Armed Forces Officers,admiral
0,110,Commissioned Armed Forces Officers,brigadier army
0,110,Commissioned Armed Forces Officers,cadet officer armed forces
0,110,Commissioned Armed Forces Officers,captain air force
0,110,Commissioned Armed Forces Officers,captain army
...,...,...,...
435,9629,Refuse Workers and Other Elementary Workers,collector ticket
435,9629,Refuse Workers and Other Elementary Workers,poster bill
435,9629,Refuse Workers and Other Elementary Workers,sticker bill
435,9629,Refuse Workers and Other Elementary Workers,usher


In [6]:
from polyfuzz import PolyFuzz

abbreviation_dict = {
    'Sr.': 'Senior',
    'Jr.': 'Junior',
    'VP': 'Vice President',
    'Mgr': 'Manager',
    'Dir': 'Director',
    'HR': 'Human Resources',
    'PA': 'Personal Assistant',
    'PR': 'Public Relations',
    'QA': 'Quality Assurance',
}

def replace_abbreviations(title, abbreviation_dict):
    words = title.split()  # Tokenize the title into words
    transformed_words = [abbreviation_dict.get(word, word) for word in words]  # Replace abbreviations
    return ' '.join(transformed_words)  # Join words back together

merged_df['translated_title'] = merged_df['translated_title'].apply(replace_abbreviations, abbreviation_dict=abbreviation_dict)

from_list = list(merged_df['translated_title'])
to_list = list(df_code_exploded['english title'].unique())

model = PolyFuzz("TF-IDF")
model.match(from_list, to_list)
inspect_df = model.get_matches()

/home/quangtien/miniconda3/envs/visa/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
inspect_df

,From,To,Similarity
0,Panel & Paint Technician,technician drain,0.563
1,Teacher in crafts and technology for grades 7-...,teacher dance school,0.397
2,Consultants in Emergency Medicine - Doughiska,specialist emergency medicine,0.509
3,Senior IT Support Engineers,support it,0.599
4,Commercial Sales Representatives,representative sales,0.761
...,...,...,...
25660,Gallerato showroom sales consultant,consultant sales rental,0.468
25661,Pedestrian Marshall,marshal air,0.622
25662,Unity Gameplay Developer (f/m/d),developer video game,0.510
25663,Carpenter 4505717 | careers4a.com,carpenter,0.459


In [8]:
merged_df['matching_title'] = inspect_df['To']
merged_df['score_match'] = inspect_df['Similarity']


In [9]:
merged_df = merged_df.merge(df_code_exploded, left_on='matching_title', right_on='english title', how='left')

In [10]:
merged_df = merged_df.drop_duplicates(subset=['id'], keep='first')
merged_df

,id,top-k,alternatives,Final Prediction,translated_title,final_description,matching_title,score_match,code,description,english title
0,872828466,"7132, 7131, 7231, 9122, 7316, 7232, 3139, 3115...","7131, 7231, 9122",7132,Panel & Paint Technician,Panel & Paint Technician Panel & Paint Technic...,technician drain,0.563,7126.0,Building and Related Trades Workers (excluding...,technician drain
1,839465958,"2320, 7533, 7318, 2330, 2353, 7222, 8152, 2355...",NaN,2330,Teacher in crafts and technology for grades 7-...,Teacher in crafts and technology for grades 7-...,teacher dance school,0.397,2355.0,Teaching Professionals,teacher dance school
2,857077872,"2211, 2212, 2240, 3258, 2230, 4229, 3211, 4221...",NaN,2212,Consultants in Emergency Medicine - Doughiska,Consultants in Emergency Medicine - Doughiska ...,specialist emergency medicine,0.509,2212.0,Health Professionals,specialist emergency medicine
3,801801567,"3512, 2522, 2356, 3511, 2511, 2153, 1330, 3513...",NaN,3512,Senior IT Support Engineers,"Senior IT Support Engineers My Client, who has...",support it,0.599,3512.0,Information and Communications Technicians,support it
4,855162927,"3322, 2433, 2434, 5243, 5223, 5244, 5242, 1221...",NaN,3322,Commercial Sales Representatives,"Commercial Sales Representatives Jobbtitel: ""C...",representative sales,0.761,3322.0,Business and Administration Associate Professi...,representative sales
...,...,...,...,...,...,...,...,...,...,...,...
25751,862998979,"3433, 5223, 5243, 5242, 5211, 3432, 3322, 5244...",NaN,5223,Gallerato showroom sales consultant,Gallerato showroom sales consultant Descriptio...,consultant sales rental,0.468,5249.0,Sales Workers,consultant sales rental
25752,793143661,"9312, 9313, 9622, 9510, 8343, 9331, 9214, 8342...","9510, 9613, 9329",5419,Pedestrian Marshall,Pedestrian Marshall Pedestrial Marshall / Yard...,marshal air,0.622,110.0,Commissioned Armed Forces Officers,marshal air
25753,725881734,"2513, 2519, 2512, 2166, 2514, 2431, 3435, 2522...",NaN,2166,Unity Gameplay Developer (f/m/d),Unity Gameplay Developer (f/m/d) Unity Gamepla...,developer video game,0.510,2513.0,Information and Communications Technology Prof...,developer video game
25754,880528881,"7115, 7522, 7111, 9313, 7119, 7121, 7123, 9312...",NaN,7115,Carpenter 4505717 | careers4a.com,Carpenter 4505717 | careers4a.com What: jo...,carpenter,0.459,7115.0,Building and Related Trades Workers (excluding...,carpenter


In [11]:
df_inspecting = merged_df[merged_df['score_match'] >= 0.9][merged_df['Final Prediction'] != merged_df['code']][merged_df['translated_title'].str.len() >= merged_df['matching_title'].str.len()]
df_inspecting

/tmp/ipykernel_1135418/196440977.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_inspecting = merged_df[merged_df['score_match'] >= 0.9][merged_df['Final Prediction'] != merged_df['code']][merged_df['translated_title'].str.len() >= merged_df['matching_title'].str.len()]
/tmp/ipykernel_1135418/196440977.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_inspecting = merged_df[merged_df['score_match'] >= 0.9][merged_df['Final Prediction'] != merged_df['code']][merged_df['translated_title'].str.len() >= merged_df['matching_title'].str.len()]


,id,top-k,alternatives,Final Prediction,translated_title,final_description,matching_title,score_match,code,description,english title
112,724447679,"2511, 2521, 2529, 3314, 2413, 2120, 4312, 2421...","2521, 2523, 2631",2511,Data Management Consultant,Data Management Consultant Project information...,consultant management,0.911,2421.0,Business and Administration Professionals,consultant management
121,839346460,"3322, 3311, 2434, 2433, 2413, 4312, 5244, 3321...",NaN,2433,SALES REPRESENTATIVE,SALES REPRESENTATIVE Trader for Platon Finance...,representative sales,1.000,3322.0,Business and Administration Associate Professi...,representative sales
313,804870659,"2161, 7111, 5153, 2142, 3123, 3112, 1323, 9313...","3112, 2142, 2149",2165,Building Surveyor,Building Surveyor My client is looking for a B...,surveyor building,1.000,3112.0,Science and Engineering Associate Professionals,surveyor building
451,889671981,"4416, 4229, 4321, 3341, 4110, 3344, 4313, 3252...","4222, 4226, 4419",3252,Ward Clerk,Ward Clerk Please switch to a desktop device b...,clerk ward,1.000,4229.0,Customer Services Clerks,clerk ward
495,754092194,"1330, 2434, 3512, 2433, 7422, 2522, 2511, 3511...","2433, 1330, 1221",3322,ICT Account Manager,ICT Account Manager Job Context & Duties Perfo...,manager account ict,1.000,2434.0,Business and Administration Professionals,manager account ict
...,...,...,...,...,...,...,...,...,...,...,...
25002,747953035,"2262, 3213, 1420, 1344, 1342, 5222, 5221, 1324...","1420, 2262, 3213",1342,Pharmacist Manager,Pharmacist Manager Job description Pharmacist ...,pharmacist,0.906,2262.0,Health Professionals,pharmacist
25228,780527236,"9329, 1321, 3122, 8121, 2141, 3135, 8211, 4322...","3111, 3112",2142,Manufacturing Data Engineer,Manufacturing Data Engineer Manufacturing Data...,engineer manufacturing,0.939,2141.0,Science and Engineering Professionals,engineer manufacturing
25413,830146085,"5322, 5321, 5162, 5329, 5312, 5169, 3221, 3412...","5152, 9111, 5322",5162,Personal assistant,"Personal assistant Personal assistant, Kuusiok...",assistant personal,1.000,3343.0,Business and Administration Associate Professi...,assistant personal
25515,804601060,"2423, 3321, 3339, 3333, 2412, 2132, 4221, 4312...","2433, 3322, 2423",2511,Business Consultant,Business Consultant EY GDS (Global Delivery Se...,consultant business,1.000,2421.0,Business and Administration Professionals,consultant business


In [12]:
merged_df.loc[df_inspecting.index, 'Final Prediction'] = df_inspecting['code']

In [31]:
merged_df.to_csv('top_k_prediction/fuzzy_match_merged_predictions_latest.csv', index=False)

In [13]:
out_columns = ['id', 'Final Prediction']
out_df = merged_df[out_columns]
out_df['id']= out_df['id'].astype(str)
out_df['Final Prediction']= out_df['Final Prediction'].astype(str)
out_df['Final Prediction'] = out_df['Final Prediction'].replace(['110', '210', '310'], ['0110', '0210', '0310'])

/tmp/ipykernel_1135418/4269472091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_df['id']= out_df['id'].astype(str)
/tmp/ipykernel_1135418/4269472091.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out_df['Final Prediction']= out_df['Final Prediction'].astype(str)
/tmp/ipykernel_1135418/4269472091.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [14]:
out_df[out_df['Final Prediction'].str.len() != 4]

,id,Final Prediction


In [15]:
out_df[out_df['id'] == '854963219']

,id,Final Prediction
168,854963219,1114


In [16]:
out_df.to_csv('top_k_prediction/classification.csv', index=False, header=False)

In [27]:
out_df[out_df['Final Prediction'].isnull()]


,id,Final Prediction
